In [ ]:
import numpy as np
from optimas.explorations import Exploration
from optimas.core import VaryingParameter, Objective, Parameter
from optimas.generators import RandomSamplingGenerator
from optimas.evaluators import FunctionEvaluator


def eval_func_sf_moo(input_params, output_params):
    """Example multi-objective function."""
    x1 = input_params["x1"]
    x2 = input_params["x2"]
    result = -(x1 + 10 * np.cos(x1)) * (x2 + 5 * np.cos(x2))
    output_params["f1"] = result
    output_params["f2"] = result * 2
    output_params["p1"] = np.sin(x1) + np.cos(x2)


var1 = VaryingParameter("x1", 0.0, 5.0)
var2 = VaryingParameter("x2", -5.0, 5.0)
par1 = Parameter("p1")
obj1 = Objective("f1", minimize=True)
obj2 = Objective("f2", minimize=False)

gen = RandomSamplingGenerator(
    varying_parameters=[var1, var2],
    objectives=[obj1, obj2],
    analyzed_parameters=[par1],
)
ev = FunctionEvaluator(function=eval_func_sf_moo)
exploration = Exploration(
    generator=gen,
    evaluator=ev,
    max_evals=50,
    sim_workers=1,
    exploration_dir_path="./exploration",
    libe_comms="threads",  #this is only needed to run on a Jupyter notebook.
)

# Run exploration.
exploration.run()

In [ ]:
from optimas.diagnostics import ExplorationDiagnostics

diags = ExplorationDiagnostics(exploration)

In [ ]:
# Build one model for each objective and analyzed parameter.
f1_model = diags.build_gp_model("f1")
f2_model = diags.build_gp_model("f2")
p1_model = diags.build_gp_model("p1", minimize=False)

In [ ]:
# plot model for `f1`.
fig, ax1 = f1_model.plot_contour(mode="both", figsize=(6, 3), dpi=300)

In [ ]:
# plot 1D slice of `f1`.
fig, ax1 = f1_model.plot_slice("x1", figsize=(6, 3), dpi=300)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

fig = plt.figure(figsize=(10, 3), dpi=300)
gs = GridSpec(1, 3, wspace=0.4)

# plot model for `f1`.
fig, ax1 = f1_model.plot_contour(
    pcolormesh_kw={"cmap": "GnBu"},
    subplot_spec=gs[0, 0],
)

# Get and draw top 3 evaluations for `f`
df_top = diags.get_best_evaluations(top=3, objective="f1")
ax1.scatter(df_top["x1"], df_top["x2"], c="red", marker="x")

# plot model for `f2`
fig, ax2 = f2_model.plot_contour(
    pcolormesh_kw={"cmap": "OrRd"},
    subplot_spec=gs[0, 1],
)

# plot model for `p1`
fig, ax3 = p1_model.plot_contour(
    pcolormesh_kw={"cmap": "PuBu"},
    subplot_spec=gs[0, 2],
)

In [ ]:
# Evaluate model for each point in the history
mean, sem = f1_model.evaluate_model(diags.history)
min_f, max_f = np.min(diags.history["f1"]), np.max(diags.history["f1"])

# Make plot
fig, ax = plt.subplots(figsize=(5, 4), dpi=300)
ax.errorbar(diags.history["f1"], mean, yerr=sem, fmt="o", ms=4, label="Data")
ax.plot([min_f, max_f], [min_f, max_f], color="k", ls="--", label="Ideal correlation")
ax.set_xlabel("Observations")
ax.set_ylabel("Model predictions")
ax.legend(frameon=False)